In [23]:
import numpy as np

### Data I/O

In [24]:
data = open("shakespeare.txt", "r").read()
chars = list(set(data))
print(chars)
print(len(data))
data_size, vocab_size = len(data), len(chars)
char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i, ch in enumerate(chars)}

['T', 'O', 'U', '$', '-', 'S', ',', 'P', '!', 'F', 'w', 'B', ';', 'W', 'a', 'X', 'R', 'b', 'Z', 'p', 't', 'V', '.', 'n', 'h', 'k', 'Y', 'o', '3', 'A', 's', 'v', 'e', '[', 'j', 'H', 'd', 'q', '&', 'D', 'K', 'C', ' ', 'l', 'c', 'g', 'N', 'y', 'z', 'G', 'r', 'E', 'm', '\n', 'x', "'", 'Q', ':', 'J', ']', 'M', 'f', 'L', 'I', 'u', '?', 'i']
4573338


### Hyperparameter

In [25]:
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

### Model Parameter

In [26]:
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
Why = np.random.randn(vocab_size, hidden_size) * 0.01
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))
print(Wxh.shape)
print(Whh.shape)
print(Why.shape)

(100, 67)
(100, 100)
(67, 100)


In [27]:
# p = 0
# print(data[p:p+seq_length])
# print(data[p+1:p+seq_length+1])
# inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
# targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
# hprev = np.zeros((hidden_size, 1))

# print(inputs)
# print(targets)

In [166]:
def lossFunction(inputs, targets, hprev):
    xs, hs, ys, ps = {}, {}, {}, {}
    loss = 0
    hs[-1] = np.copy(hprev)
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1
        
        hs[t] = np.tanh(np.dot(Whh, hs[t-1]) + np.dot(Wxh, xs[t]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) 
        loss += -np.log(ps[t][targets[t], 0])
#     print("---XS---",xs)
        #print("Probs",ps[t])
    print("HS:",len(hs[0]))
#     print("HS:",hs)

    
    mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    
    for t in reversed(range(len(inputs))):
#         print("back iter:",t)
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]        
        
loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFunction(inputs, targets, hprev)  

# print(dWxh.shape)
# print(dWhh.shape)
# print(dWhy.shape)


HS: 100


In [39]:
def sample(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [42]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while(n<1000):
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
#     if n % 100 == 0:
#         sample_ix = sample(hprev, inputs[0], 200)
#         txt = ''.join(ix_to_char[ix] for ix in sample_ix)
#         print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFunction(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 10 == 0: 
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter 

iter 0, loss: 105.075825
iter 10, loss: 105.605261
iter 20, loss: 105.573109
iter 30, loss: 105.394430
iter 40, loss: 105.188741
iter 50, loss: 104.993643
iter 60, loss: 104.724870
iter 70, loss: 104.441059
iter 80, loss: 104.167545
iter 90, loss: 103.963470
iter 100, loss: 103.762445
iter 110, loss: 103.656251
iter 120, loss: 103.408178
iter 130, loss: 103.143661
iter 140, loss: 102.933745
iter 150, loss: 102.723364
iter 160, loss: 102.479413
iter 170, loss: 102.267698
iter 180, loss: 102.026273
iter 190, loss: 101.835425
iter 200, loss: 101.568291
iter 210, loss: 101.353747
iter 220, loss: 101.151835
iter 230, loss: 100.941359
iter 240, loss: 100.661483
iter 250, loss: 100.419059
iter 260, loss: 100.149840
iter 270, loss: 99.890724
iter 280, loss: 99.645581
iter 290, loss: 99.349035
iter 300, loss: 99.148149
iter 310, loss: 98.858901
iter 320, loss: 98.596433
iter 330, loss: 98.333540
iter 340, loss: 98.053045
iter 350, loss: 97.799420
iter 360, loss: 97.577901
iter 370, loss: 97.349

### Back Propagation

#### error = (Actual Output - Predicted Output)^2
#### delta w = learning rate * (de/dw)
#### W = w + delta w

### Character Tokenizations  

In [35]:
p = 0
n = 0
X = []
Y = []
while (n < 100):
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    X.append(inputs)
    Y.append(targets)
    p += seq_length
    n += 1
print(len(X))
print(len(Y))

100
100


In [36]:
import re
with open('X_100.txt', 'w') as fx:
    for item in X:
        string = re.sub('[\[\]]','',repr(item))
        fx.write("%s\n" % string)
with open('Y_100.txt', 'w') as fy:
    for item in Y:
        string = re.sub('[\[\]]','',repr(item))
        fy.write("%s\n" % string)